In [ ]:
# @title
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from scipy import stats

# Tarea 2 : Estadisticos de Radiación
## Integrantes:
Juan José García gil

Juan Sebastián Gallón

Leonardo Ortega Trujillo

Darío Penagos

Estefanía Zapata Valderrama

---

### Objetivo: Determinar si las muestras radiactivas presentan actividad significativa respecto a la radiación de fondo.

---

### Marco Teorico

Los tubos escintiladores (o detectores de escintilación) son instrumentos esenciales para la detección de radiación ionizante (partículas $\alpha$, $\beta$, $\gamma$, o rayos X). Su funcionamiento se basa en un escintilador(material luminiscente) que absorbe la energía de la radiación y emite fotones (por fluorescencia o fosforescencia).Esta luz, que es proporcional a la energía incidente, es amplificada por un fotomultiplicador.El fotomultiplicador convierte los fotones en electrones y los multiplica exponencialmente para generar una señal eléctrica medible.

El dispositivo es sensible a la radiación de fondo (producida, por ejemplo, por rayos cósmicos) y a decaimientos radiactivos en muestras. Los eventos de decaimiento radiactivo, caracterizados por un alto número de sucesos con probabilidades muy bajas de ocurrir en un intervalo de tiempo ($p \ll 1$), se ajustan a la estadística de Poisson.

La función de probabilidad de Poisson es:

$$
P(k; \lambda) = \frac{\lambda^k e^{-\lambda}}{k!}
$$

donde $k$ es el número de eventos observados y $\lambda$ es la media esperada de eventos por intervalo.Esta distribución permite modelar la radiación de fondo y, mediante pruebas de hipótesis (como la $t$ de Welch) y el criterio de chi-cuadrado ($\chi^2$), discriminar estadísticamente la señal de muestras activas con un nivel de confianza del 95% ($\alpha = 0.05$).



### Metodología


La metodología se centró en un análisis estadístico comparativo de la radiación de fondo frente a las muestras de Torio y Americio, empleando un Tubo Escintilador y una placa Arduino como detectores alternativos. Los pasos experimentales se implementaron y analizaron en un cuaderno Jupyter.

 1. Medición Experimental

* Se determinó un intervalo de tiempo promedio para 3 conteos (con 10 mediciones).
* Se realizaron 30 mediciones de conteo con una duración igual a ese intervalo, primero para la radiación de fondo y luego para las muestras de Torio y Americio.

2. Análisis Descriptivo y de Distribución

* Se calculó la media muestral ($\bar{x}$) para cada conjunto de datos. Esta media se usó como el parámetro $\lambda$ en la fórmula de la distribución de Poisson.
* Se generó un histograma de conteos vs. frecuencia.
* El histograma fue ajustado con la función de Poisson y la bondad del ajuste fue determinada usando el criterio de chi-cuadrado ($\chi^2$).


3. Prueba de Hipótesis (Actividad de Muestras)

* Para determinar si las muestras eran activas con un 95% de confianza ($\alpha = 0.05$), se comparó la media de la muestra ($\bar{x}_s$) contra la media del fondo ($\bar{x}_f$).
* Se utilizó la prueba $t$ de Welch para varianzas desiguales, calculando el estadístico $t$:

    $$
    t = \frac{\bar{x}_s - \bar{x}_f}{SE}
    $$

    donde $SE$ es el error estándar combinado.
* Se calcularon los $p$-valores y los intervalos de confianza al 95%. La hipótesis nula ($H_0$: la muestra no presenta actividad) se rechazó si el $p$-valor era menor que $\alpha = 0.05$.

### Análisis Descriptivo y de Distribución

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ruta0 = '/content/drive/MyDrive/Data/datos_experimental.csv'
import pandas as pd
tubo = pd.read_csv(ruta0)

ruta1 = '/content/drive/MyDrive/Data/Arduino_data.csv'
import pandas as pd
arduino = pd.read_csv(ruta1)

In [ ]:
def value(material, instrumento):
  X = instrumento[material]
  X = X.to_numpy()
  return X

In [ ]:
Instrumentos = [arduino, tubo]
Materiales = ['fondo', 'Torio', 'Americio']

In [ ]:
#tubo escintilador
t_fondo = value(Materiales[0], Instrumentos[1])
t_torio = value(Materiales[1], Instrumentos[1])
t_americio = value(Materiales[2], Instrumentos[1])

#arduino
a_fondo = value(Materiales[0], Instrumentos[0])
a_torio = value(Materiales[1], Instrumentos[0])
a_americio = value(Materiales[2], Instrumentos[0])

#Datos
Dat = [t_fondo, t_torio, t_americio, a_fondo, a_torio, a_americio]

In [ ]:
#Media
def Med(X):
  M = sum(X)/len(X)
  return float(M)

#Varianza
def Var(X, m):
  n = []
  for i in range(len(X)):
    n.append((X[i] - m)**2)
  num = sum(n)
  var = num/(len(X)-1)
  return float(var)

#Diferencial de medias
def dm(s_fondo, s2):
  f1 = s2
  f2 = s_fondo
  r = np.sqrt(f1 + f2)
  return float(r*np.sqrt(1/30))

#Estadísitico t tipo Welch
def welch(Ms, Mf, SE):
  return float((Ms - Mf)/SE)

In [ ]:
medias = []
for i in range(len(Dat)):
  medias.append(Med(Dat[i]))

varianza = []
for i in range(len(Dat)):
  varianza.append(Var(Dat[i], medias[i]))

SEa = []
SEt = []
for i in range(1, 3):
  SEt.append(dm(varianza[0], varianza[i]))
for i in range(4, len(Dat)):
  SEa.append(dm(varianza[3], varianza[i]))

ta = []
tt = []
for i in range(2):
  ta.append(welch(medias[i+1], medias[0], SEa[i]))
for i in range(2):
  tt.append(welch(medias[i+4], medias[3], SEt[i]))

In [ ]:
#Grados de libertad
def v(s, sf, SE):
  num = SE**4
  den1 = ((s**2)/30)**2
  den2 = ((sf**2)/30)**2
  den = (den1 + den2)*(1/29)
  return float(num/den)

In [ ]:
li_t = []
li_a = []
for i in range(2):
  li_t.append(v(varianza[i+1], varianza[0], SEt[i]))
for i in range(2):
  li_a.append(v(varianza[i+4], varianza[3], SEa[i]))

In [ ]:
#p valores
p_va = []
p_vt = []
for i in range(2):
  p_va.append(1-stats.t.cdf(ta[i], df = li_a[i]))
for i in range(2):
  p_vt.append(1-stats.t.cdf(tt[i], df = li_t[i]))

#valor crítico
alpha = 0.05

tcrit_va = []
tcrit_vt = []
for i in range(2):
  tcrit_va.append(stats.t.ppf(1-alpha, df = li_a[i]))
for i in range(2):
  tcrit_vt.append(stats.t.ppf(1-alpha, df = li_t[i]))

#Intervalos de confianza
CIt = []
CIa = []

for i in range(2):
    # Tubo: comparaciones con fondo
    CIt.append((
        float((medias[i+1] - medias[0])) - float(tcrit_vt[i]*SEt[i]),
        float((medias[i+1] - medias[0])) + float(tcrit_vt[i]*SEt[i])
    ))

for i in range(2):
    # Arduino: comparaciones con fondo
    CIa.append((
        float((medias[i+4] - medias[3])) - float(tcrit_va[i]*SEa[i]),
        float((medias[i+4] - medias[3])) + float(tcrit_va[i]*SEa[i])
    ))

print('intervalo de confianza Torio (Arduino) ' + str(CIa[0]))
print('intervalo de confianza Americio (Arduino) ' + str(CIa[0]))
print('intervalo de confianza Torio (Tubo) ' + str(CIt[0]))
print('intervalo de confianza Americio (Tubo) ' + str(CIt[0]))

intervalo de confianza Torio (Arduino) (-2.371207210285085e+18, 2.371207210285085e+18)
intervalo de confianza Americio (Arduino) (-2.371207210285085e+18, 2.371207210285085e+18)
intervalo de confianza Torio (Tubo) (-1.2532855860842881e+20, 1.2532855860842881e+20)
intervalo de confianza Americio (Tubo) (-1.2532855860842881e+20, 1.2532855860842881e+20)


In [ ]:
#Decisión con respecto a la Hipótesis nula de que la muestra no presenta actividad respecto al fondo

#Tubo
for i in range(2):
  if p_vt[i] < alpha:
    print(str(p_vt[i]) + ' Se descarta la hipótesis nula')
  else:
    print(str(p_vt[i]) + ' No se descarta la hipótesis nula')

#Arduino
for i in range(2):
  if p_va[i] < alpha:
    print(str(p_va[i]) + ' Se descarta la hipótesis nula')
  else:
    print(str(p_va[i]) + ' No se descarta la hipótesis nula')

0.3851222280240174 No se descarta la hipótesis nula
9.067768084181083e-07 Se descarta la hipótesis nula
0.3871177912157986 No se descarta la hipótesis nula
0.49666778777732234 No se descarta la hipótesis nula


### Conclusiones


| Instrumento | Muestra | $p$-valor | Resultado (vs. $H_0$) |
| :---: | :---: | :---: | :---: |
| Tubo Escintilador | **Torio** | $0.3851$ | No se descarta $H_0$ |
| Tubo Escintilador | **Americio** | **$9.06 \times 10^{-7}$** | **Se descarta $H_0$** |
| Arduino | **Torio** | $0.3871$ | No se descarta $H_0$ |
| Arduino | **Americio** | $0.4967$ | No se descarta $H_0$ |

---
El análisis estadístico comparó la actividad de las muestras de Torio y Americio contra la radiación de fondo, utilizando el criterio del $p$-valor:

1. Actividad Significativa: La única prueba que permitió descartar la hipótesis nula y concluir que existe una actividad estadísticamente significativa fue la del Americio medida con el tubo Escintilador ($p \ll 0.05$).

2. La placa Arduino no logró detectar actividad significativa en ninguna de las dos muestras (Torio: $p \approx 0.387$; Americio: $p \approx 0.497$), ya que ambos $p$-valores son mayores que $0.05$.

3. El Tubo Escintilador, a pesar de su mayor sensibilidad, tampoco detectó actividad significativa para el Torio ($p \approx 0.385$).

La conclusión es que las muestras radiactivas muestran actividad respecto al fondo, pero el caso particular del Torio o la medición con Arduino, donde la actividad no es significativa, sugiere que la radiación involucrada podría no ser lo suficientemente energética o intensa para ser capturada de forma consistente por el Tubo Escintilador o simplemente cae dentro del rango de fluctuación del fondo para el Arduino.